# Optimization Engine

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time
import warnings
warnings.filterwarnings('ignore')

# Load the Feature engineered Dataset and parse the timestamp column
df_fe = pd.read_csv('dataset/HVAC Energy Data Feature Engineered.csv')
timestamp_column = 'Local Time (Timezone : GMT+8h)'
df_fe[timestamp_column] = pd.to_datetime(df_fe[timestamp_column])
df_fe = df_fe.set_index(timestamp_column)

## Define optimization parameters

In [3]:
# Get variable ranges from training data
chilled_water_min = df_fe['Chilled Water Rate (L/sec)'].min()
chilled_water_max = df_fe['Chilled Water Rate (L/sec)'].max()
# Corrected column name
cooling_water_min = df_fe['Cooling Water Temperature (C)'].min()
cooling_water_max = df_fe['Cooling Water Temperature (C)'].max()


print(f"\n✓ Chilled Water Rate range: [{chilled_water_min:.2f}, {chilled_water_max:.2f}] L/sec")
print(f"✓ Cooling Water Temperature range: [{cooling_water_min:.2f}, {cooling_water_max:.2f}] °C")

# Constraint thresholds
TEMP_MAX = 28.0
TEMP_MIN = 20.0
TEMP_TARGET = 24.0

print(f"\n✓ Temperature constraints: [{TEMP_MIN}°C, {TEMP_MAX}°C]")
print(f"✓ Target temperature: {TEMP_TARGET}°C")



✓ Chilled Water Rate range: [72.40, 141.50] L/sec
✓ Cooling Water Temperature range: [25.80, 36.20] °C

✓ Temperature constraints: [20.0°C, 28.0°C]
✓ Target temperature: 24.0°C


## Demonstration on Test set

In [8]:
import joblib
try:
    energy_model = joblib.load('trained_models/xgboost_energy_model.pkl')
    print("Energy model loaded from file for optimization")
except FileNotFoundError:
    print("Error: 'xgboost_energy_model.pkl' not found.")
    raise

X_test = pd.read_csv('dataset/X_test_optimization_engine.csv', index_col=0)

# Calculate average energy savings potential across test set
total_samples = len(X_test)
optimization_count = 0
total_savings = 0

print(f"\nTesting optimization on {total_samples} test samples...")
print("Finding energy savings opportunities...\n")

# Test on first 500 samples
for i in range(min(500, total_samples)):
    sample = X_test.iloc[i].values
    baseline_energy = energy_model.predict([sample])[0]

    # Test slight adjustments (simulating optimization)
    best_energy = baseline_energy
    for adj in [-0.05, -0.02, 0, 0.02, 0.05]:
        test_sample = sample.copy()
        test_sample[0] = np.clip(sample[0] + adj, 0, 1)
        pred_energy = energy_model.predict([test_sample])[0]

        if pred_energy < best_energy:
            best_energy = pred_energy

    savings = baseline_energy - best_energy
    if savings > 0:
        optimization_count += 1
        total_savings += savings

avg_savings = total_savings / optimization_count if optimization_count > 0 else 0
total_energy_baseline = sum(energy_model.predict(X_test[:500]))
total_energy_optimized = total_energy_baseline - total_savings
percent_reduction = (total_savings / total_energy_baseline) * 100

print("OPTIMIZATION ENGINE RESULTS")
print(f"\nTest Set Analysis (500 samples):")
print(f"  • Samples with optimization potential: {optimization_count}/{min(500, total_samples)}")
print(f"  • Average energy savings per optimized sample: {avg_savings:.3f} kWh")
print(f"  • Total baseline energy: {total_energy_baseline:.2f} kWh")
print(f"  • Total optimized energy: {total_energy_optimized:.2f} kWh")
print(f"  • Total energy saved: {total_savings:.2f} kWh")
print(f"  • Overall energy reduction: {percent_reduction:.2f}%")

Energy model loaded from file for optimization

Testing optimization on 2722 test samples...
Finding energy savings opportunities...

OPTIMIZATION ENGINE RESULTS

Test Set Analysis (500 samples):
  • Samples with optimization potential: 404/500
  • Average energy savings per optimized sample: 5.563 kWh
  • Total baseline energy: 56480.29 kWh
  • Total optimized energy: 54232.77 kWh
  • Total energy saved: 2247.52 kWh
  • Overall energy reduction: 3.98%


In [ ]:
# Cost savings calculation (assuming $0.10 per kWh)
cost_per_kwh = 0.10
annual_savings = total_savings * (365 * 24 / 500)  # Extrapolate to annual
cost_savings = annual_savings * cost_per_kwh

print(f"\nFINANCIAL IMPACT (Estimated):")
print(f"  • Cost per kWh: ${cost_per_kwh:.2f}")
print(f"  • Extrapolated annual energy savings: {annual_savings:.2f} kWh")
print(f"  • Extrapolated annual cost savings: ${cost_savings:.2f}")


FINANCIAL IMPACT (Estimated):
  • Cost per kWh: $0.10
  • Extrapolated annual energy savings: 39376.54 kWh
  • Extrapolated annual cost savings: $3937.65
